In [2]:
import json
import openai
import requests
from flujo_de_dialogo import ventana_de_historico, guardar_historico
from calcular_tokens import num_tokens_from_messages, num_tokens_from_string
from parse import parse_class_predictions, parse_proba_predictions
from open_ai_utils import simular_respuesta_generativa
from load_store_utils import cargar_modelo 
from anomaly_detection import predict_anomalies, cargar_modelos_anomaly_detection
from anomaly_detection import nueva_consulta_anomaly_detection
from flujo_de_dialogo import pregunta_sobre_consulta_anterior_AD, continuar_conversacion_AD

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


import os
from dotenv import load_dotenv
from load_store_utils import cargar_lote_datos
from open_ai_utils import enviar_promt_chat_completions_mode
import warnings

# Ignorar todos los warnings
warnings.filterwarnings('ignore')

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

In [3]:
tools = [
    {
    "type": "function",
    "function": {
        "name": "cargar_lote_datos",
        "description": "Carga un lote de datos de un archivo CSV basado en el número de muestras especificadas cada muestra representa 1 hora.\
                        Si no te especifican un numero exacto: m_samples = 5",
        "parameters": {
            "type": "object",
            "properties": {
                "m_samples": {
                    "type": "integer",
                    "description": "Número de muestras a cargar"
                },
                "formato" : {
                    "type": "string",
                    "enum": ["DataFrame", "numpy"],
                    "description": "Formato en el que se devuelven los datos, DataFrame por defecto"
                }
            },
            "required": ["m_samples"]
        }
    }
}
]

In [4]:
conversacion = True
modelos_cargados = False
contador_interacciones = 1
nueva_consulta = True
historico_completo = ''

simular_respuesta_generativa('AGENTE:\nHola! en que te puedo ayudar hoy?\n\n')
while conversacion: 
    
    if nueva_consulta==True: 
        consulta_usuario = str(input())
        simular_respuesta_generativa(f'PROMPT_USUARIO_{contador_interacciones}:\n{consulta_usuario}\n\n')


        extraccion_argumentos = [
        {'role': 'system', 'content': f'Tu objetivo es extraer los argumentos necesarios para ejecutar la función que te he pasado en tools'}, 
        {'role': 'user', 'content': f'{consulta_usuario}'} ]

        argumentos_extraidos_del_llm = enviar_promt_chat_completions_mode(
                mensaje=extraccion_argumentos, 
                funciones= tools, 
                forzar_funciones= {"type": "function", "function": {"name": "cargar_lote_datos"}}, 
                aleatoriedad= 0, 
                probabilidad_acumulada=1)

        simular_respuesta_generativa(f'AGENTE:\nEl número de instancias a cargar es de {argumentos_extraidos_del_llm["m_samples"]}\n')

        # cargando datos
        load_data_params = {'name': 'HVAC_test_processeded.csv' , 'ruta_datos':'../data/Anomaly_detection/processed_data/'}
        try: 
            data = cargar_lote_datos(**argumentos_extraidos_del_llm, **load_data_params)
        except TypeError as e: 
            print(e)
        #display(data)

        columnas_base = ['T_Supply', 'T_Return', 'SP_Return', 'T_Saturation', 'T_Outdoor',
                         'RH_Supply', 'RH_Return', 'RH_Outdoor', 'Energy', 'Power']

        datos_base = data[columnas_base]

        data_base_text = datos_base.to_markdown()
        simular_respuesta_generativa(
            f'AGENTE:\nLos datos han sido cargados con exito! Aqui tienes la lectura de datos de tu consulta:\n\n{data_base_text}\n\n')

        # guardamos también la descripcion estadistica del los datos
        data_descr = datos_base.describe()
        data_descr_text = data_descr.to_markdown()
        # simular_respuesta_generativa(data_descr_text)
        # print('\n\n')

        # Cargando modelos
        if modelos_cargados== False: 

            simular_respuesta_generativa('AGENTE:\nVoy a cargar los modelos!...\n')
            scaler, models = cargar_modelos_anomaly_detection(ruta= './models/anomaly_detection/')
            simular_respuesta_generativa('\nLos modelos han sido cargados con exito!\n\n')
            modelos_cargados = True

        # escalado de datos
        simular_respuesta_generativa('AGENTE:\nVoy a escalar los datos para hacer inferencia sobre las lecturas...\n\n')
        X_test_scl = scaler.transform(data)
        X_test_scl_df= pd.DataFrame(X_test_scl, columns= scaler.feature_names_in_, index= data.index)
        simular_respuesta_generativa('Los datos se han escaldo correctamente!\n\n')
        
        # predicciones del modelo
        anomaly_predictions_df = predict_anomalies(models= models, scaled_data=X_test_scl_df)
        anomaly_predictions_text_df = anomaly_predictions_df['anomaly'].to_markdown()
        
        simular_respuesta_generativa('AGENTE:\nAqui tienes las predicciones para las lecturas: 0 lectura normal, 1 lectura inusual\n\n')
        simular_respuesta_generativa(anomaly_predictions_text_df)

        simular_respuesta_generativa('\n\nVoy a procesar tu consulta y generar un informe...\n\n')
        # enviamos primera consulta al agente para que genere el informe
        max_tokens_respuesta = 1000
        respuesta = nueva_consulta_anomaly_detection(
            consulta_usuario=consulta_usuario,
            datos_base=data_base_text, 
            descr_datos=data_descr_text, 
            predicciones=anomaly_predictions_text_df, 
            max_tokens_respuesta=max_tokens_respuesta)
        
        
        simular_respuesta_generativa('AGENTE:\nAqui tienes tu informe:\n\n')
        simular_respuesta_generativa(respuesta)

    # actualizamos el historico y el contador de interacciones
    historico_completo, contador_interacciones = guardar_historico(
            historico=                 historico_completo, 
            mensaje_usuario=           consulta_usuario,
            respuesta_sistema=         respuesta, 
            contador_interacciones=    contador_interacciones )
    
    # cogemos nuestra ventana de historico
    historico_parcial = ventana_de_historico(historico_completo=historico_completo)
    
    # continuamos chat sobre la lectura o nueva consulta
    simular_respuesta_generativa('\n\nAGENTE:\n ¿Tienes alguna otra consulta?\n\n')
    seguir_conversando = str(input())
    continuar_conversacion, nueva_consulta = continuar_conversacion_AD(seguir_conversando)        

    # si el usuario tiene preguntas sobre la vieja consulta
    while continuar_conversacion == True and nueva_consulta== False:
        respuesta = pregunta_sobre_consulta_anterior_AD(
            consulta_usuario=seguir_conversando, 
            historico= historico_parcial, 
            datos_base=datos_base, 
            datos_descr=data_descr, 
            predicciones=anomaly_predictions_text_df
            ) 
        simular_respuesta_generativa(respuesta)
        
        # actualizamos el historico y el contador de interacciones
        historico_completo, contador_interacciones = guardar_historico(
            historico=                 historico_completo, 
            mensaje_usuario=           consulta_usuario,
            respuesta_sistema=         respuesta, 
            contador_interacciones=    contador_interacciones )
    
        # cogemos nuestra ventana de historico
        historico_parcial = ventana_de_historico(historico_completo=historico_completo)
    
        # continuamos chat sobre la lectura o nueva consulta
        simular_respuesta_generativa('\n\nAGENTE:\n ¿Tienes alguna otra consulta?\n\n')
        seguir_conversando = str(input())
        continuar_conversacion, nueva_consulta = continuar_conversacion_AD(seguir_conversando)

    historico_completo, contador_interacciones = guardar_historico(
            historico=                 historico_completo, 
            mensaje_usuario=           seguir_conversando,
            respuesta_sistema=         'Fin de la conversación', 
            contador_interacciones=    contador_interacciones )
    
    # si el usario no tiene preguntas o nuevas consultas se termina la conversacion
    if continuar_conversacion == False:  conversacion= False 

 
    # conversacion= False
simular_respuesta_generativa('Adios! ha sido un placer')

AGENTE:
Hola! en que te puedo ayudar hoy?

PROMPT_USUARIO_1:
dime cual es el estado de sistema para el último cuarto de día

AGENTE:
El número de instancias a cargar es de 6
AGENTE:
Los datos han sido cargados con exito! Aqui tienes la lectura de datos de tu consulta:

| Timestamp           |   T_Supply |   T_Return |   SP_Return |   T_Saturation |   T_Outdoor |   RH_Supply |   RH_Return |   RH_Outdoor |   Energy |   Power |
|:--------------------|-----------:|-----------:|------------:|---------------:|------------:|------------:|------------:|-------------:|---------:|--------:|
| 2021-04-14 21:00:00 |     18.72  |    21.9475 |        21.5 |          17.74 |        15.2 |      38.875 |      24.86  |        53    |        7 |   2.502 |
| 2021-04-14 21:30:00 |     19.27  |    20.68   |        21.5 |          18.9  |        14.7 |      39.38  |      26.66  |        54    |        0 |   0     |
| 2021-04-14 22:00:00 |     19.48  |    20.22   |        20.5 |          19.35 |        14.7 |

In [1]:
print(historico_completo)

NameError: name 'historico_completo' is not defined

In [ ]:

def pregunta_sobre_consulta_anterior_AD(consulta_usuario:str, historico:str, datos_base:str, 
                                        datos_descr:str, predicciones:str, max_tokens_respuesta:int): 
    
    '''Recibe una pregunta del usuario sobre una respuesta anterior del sistema y el agente devuelve una respuesta teniendo \
       acceso al historico y a la los datos de lectura, descripción y predicciones'''

    prompt_conversacion = [
    {'role': 'system', 'content': f'Eres un asistente de ayuda para un diagnostico de fallos en un sistema energético HVAC que responde de manera concisa sobre un \
                                    informe generado previamente. Siempre pones las unidades de las variables. El historico de la conversación: {historico}\
                                    recuerda: 1 significa valor típico y -1 valor atípico. Tu respuesta debe ser como maximo de {max_tokens_respuesta-100} tokens'}, 
                                
    {'role': 'user', 'content': f'A partir de la siguiente lectura:\n {datos_base}\n, con descripción: \n {datos_descr}\n y la siguiente \
                                  predicción: \n {predicciones}\n responde a mi consulta: \n{consulta_usuario}'}]
    
    respuesta = enviar_promt_chat_completions_mode(
            mensaje= prompt_conversacion, 
            modelo="gpt-4-1106-preview", 
            maximo_tokens=max_tokens_respuesta, 
            aleatoriedad=0.2, 
            probabilidad_acumulada=0.8)

    return respuesta
    

In [2]:


# def cargar_lote_datos(m_samples, name, ruta_datos, formato= 'DataFrame'): 
#     '''Carga un lote de m ejemplos de un archivo csv determinado de manera aleatoria, 
#        en formato dataframe o en numpy array'''

#     ruta_completa = os.path.join(ruta_datos, name)

#     lote = pd.read_csv(ruta_completa).set_index('Timestamp').iloc[-m_samples:, ::]

#     if formato=='DataFrame': return lote
#     else:                    return lote.to_numpy()

# # cargando datos
# load_data_params = {'name': 'HVAC_test_processeded.csv' , 'ruta_datos':'../data/Anomaly_detection/processed_data/'}
# try: 
#     data = cargar_lote_datos(10, **load_data_params)
# except TypeError as e: 
#    print(e)

# display(data)

# columnas_base = ['T_Supply', 'T_Return', 'SP_Return', 'T_Saturation', 'T_Outdoor',
#                         'RH_Supply', 'RH_Return', 'RH_Outdoor', 'Energy', 'Power']

# datos_base = data[columnas_base]

# display(datos_base)

In [1]:
# def continuar_conversacion_AD(respuesta_usuario):

#     tools = [
#     {
#     "type": "function",
#     "function": {
#         "name": "continuar_conversacion",
#         "description": "Debes extraer DOS PARAMETROS clasificando una respuesta del usuario; por un lado:\n\
#                         tienes que clasificar el parametro 'continuar':\n  \
#                         - True (seguir chateando) si el usuario hace una pregunta o tiene alguna petición.\n \
#                         - False(no seguir chateando) si el usuario no tiene ninguna pregunta o petición.\n \
#                         por otro lado, si 'continuar'es True; debes clasificar el parametro 'nueva_cosulta':\n  \
#                         - True (es una nueva consulta) el usuario pide una lectura sobre datos de otro momento o una nueva consulta.\n \
#                         - False(es una cuestion sobre una lectura anterior) el usuario pide explicaciones sobre una consulta anterior.\n \
#                         Ambos parametros son booleanos. Si 'continuar' es False, entonces 'nueva_consulta' también es False",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "continuar": {
#                     "type": "boolean",
#                     "description": "solo puede ser True o False"
#                             },
#                 'nueva_consulta':{
#                     "type": "boolean",
#                     "description": "solo puede ser True o False"
#                             },
#             },
#             "required": ["continuar", "nueva_consulta"]
#         }
#     }
# }
# ]

#     extraccion_argumentos = [
#         {'role': 'system', 'content': f'Tu objetivo es extraer los argumentos necesarios para ejecutar la función que te he pasado en tools'}, 
#         {'role': 'user', 'content': f'{respuesta_usuario}'} ]

#     argumentos_extraidos_del_llm = enviar_promt_chat_completions_mode(
#             mensaje=extraccion_argumentos, 
#             funciones= tools, 
#             forzar_funciones= {"type": "function", "function": {"name": "continuar_conversacion"}}, 
#             aleatoriedad= 0, 
#             probabilidad_acumulada=1)
    
#     return argumentos_extraidos_del_llm['continuar'], argumentos_extraidos_del_llm['nueva_consulta']


# respuesta_usuario_si_continuar= 'puedes explicarme por que esas lecturas se consideran anomalias?'
# respuesta_usuario_no_continuar = 'vale muchas gracias me tengo que ir'
# respuesta_nueva_consulta = 'quiero saber las lecturas del dia anterior'
# respuesta_nueva_consulta_ = 'cual es el estado del sistema energético hvac para el último cuarto de dia'
# res1, new1 = continuar_conversacion(respuesta_usuario=respuesta_usuario_si_continuar)
# res2, new2 = continuar_conversacion(respuesta_usuario=respuesta_usuario_no_continuar)
# res3, new3 = continuar_conversacion(respuesta_usuario=respuesta_nueva_consulta)
# res4, new4 = continuar_conversacion(respuesta_usuario=respuesta_nueva_consulta_)

# print(res1, new1)
# print(res2, new2)
# print(res3, new3)
# print(res4, new4)
#print(continuar_conversacion(str(input())))